# $D_S$ analysis

Let's start off with a "dumb" analysis: a binned histogram analysis using $m_{4\ell}$ as the summary statistic.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib

from hstar import c6
from inference import stat

## Computation of $D_S$

The discriminant is defined as

$$D_S = -\log \left( \frac{|\mathcal{M}_{gg\to h^{\ast}\to ZZ}|^2}{|\mathcal{M}_{gg\to ZZ}|^2} \right)$$

This variable will be computed for a generated dataset by constructing two samples out of the same set of events as the following:

1. First fully-physical $gg \to ZZ$ sample from which both its matrix element and the event weight can be morphed to $c_6$.
2. Second unphysical $gg \to h^{\ast} \to ZZ$ sample from which only its matrix element is morphed to $c_6$.

In [2]:
# read dataset
filepath = '/raven/u/taepa/mcfm/MCFM-10.3/Bin/ggZZ2e2m_all/events.csv'
events = pd.read_csv(filepath)

# full gg->ZZ sample
# note: normalizing the event weights because they are important!
lumi = 300.0
ggzz = c6.Sample(k=1.83, xs=1.4783394, events=events) # cross-section x k-factor [fb]
ggzz.sm_msq_key = 'msq_gg_sm'
ggzz.c6_msq_map = {
    -5 : 'msq_gg_c6_6',
    -1 : 'msq_gg_c6_10',
    0 : 'msq_gg_c6_11',
    1 : 'msq_gg_c6_12',
    5 : 'msq_gg_c6_16'
  }
ggzz.normalize(lumi)

# signal-only gg->h*->ZZ sample
# note: negative cross-section, because it doesn't matter!
gghzz = c6.Sample(xs=-99, events=events)
gghzz.sm_msq_key = 'msq_h_sm'
gghzz.c6_msq_map = {
    -5 : 'msq_h_c6_6',
    -1 : 'msq_h_c6_10',
    0 : 'msq_h_c6_11',
    1 : 'msq_h_c6_12',
    5 : 'msq_h_c6_16'
  }

In [3]:
ds_sm = - np.log( gghzz.msq() / ggzz.msq() )

c6_vals = np.arange(-20.0, 30.0, 10.0)
msq_gghzz_c6 = gghzz.msq(c6_vals)
msq_ggzz_c6 = ggzz.msq(c6_vals)
ds_c6 = - np.log10(msq_gghzz_c6 / msq_ggzz_c6)

TypeError: 'float' object cannot be interpreted as an integer

In [26]:
ds_bins = np.arange(-5,10.1,0.1)
ds_sm_centers = 0.5 * (ds_bins[1:] + ds_bins[:-1])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), sharex=True, height_ratios=(2,1))

h_ds_sm, _ = np.histogram(ds_sm, bins=ds_bins, weights=ggzz.nu(per_event=True))
ax1.step(ds_bins[:-1], h_ds_sm, where='post', label='SM')

h_ds_c6, _ = np.histogram(ds_c6[:, 0], bins=ds_bins, weights=ggzz.nu(c6_vals[0], per_event=True))
ax1.step(ds_bins[:-1], h_ds_c6, where='post', label=f'$c_6 = {c6_vals[0]}$')
ratio = np.divide(h_ds_c6, h_ds_sm, out=np.zeros_like(h_ds_sm), where=h_ds_sm!=0)
ax2.plot(ds_sm_centers, ratio, '--', label=f'$c_6 = {c6_vals[0]}$')

h_ds_c6, _ = np.histogram(ds_c6[:, -1], bins=ds_bins, weights=ggzz.nu(c6_vals[-1], per_event=True))
ax1.step(ds_bins[:-1], h_ds_c6, where='post', color='red', label=f'$c_6 = {c6_vals[-1]}$')
ratio = np.divide(h_ds_c6, h_ds_sm, out=np.zeros_like(h_ds_sm), where=h_ds_sm!=0)
ax2.plot(ds_sm_centers, ratio, '--', color='red', label=f'$c_6 = {c6_vals[-1]}$')

ax1.set_xlim(-5,10)
ax1.set_ylabel('Number of Events')
# ax1.set_yscale('log')
ax1.legend()

ax2.set_ylim(0.8,1.2)
ax2.set_xlabel('$m_{4\\ell}$ [GeV]')
ax2.set_ylabel('$c_6$ / SM')

plt.tight_layout()
plt.show()

In [22]:
c6_vals = np.linspace(-20.0, 20.0, 201)
nll = np.zeros_like(c6_vals)

m4l_sm, _ = np.histogram(m4l, bins=m4l_bins, weights=ggzz.nu(per_event=True))
wts_c6 = ggzz.nu(c6_vals, per_event=True)
for i, c6_val in enumerate(c6_vals):
  m4l_c6, _ = np.histogram(m4l, bins=m4l_bins, weights=wts_c6[:,i])
  nll[i] = stat.nll(m4l_sm, m4l_c6)
nll = nll - np.min(nll)

NameError: name 'm4l' is not defined

In [25]:
c6_vals = np.round(c6_vals, decimals = 1)
df = pd.DataFrame({'c6' : c6_vals, 'nll' : nll} )
df.to_csv('c6_nll_m4l.csv', index=False)